In [1]:
# noexport

!typech evaluation_utils.ipynb


[NbConvertApp] Converting notebook evaluation_utils.ipynb to python
evaluation_utils.py:413: error: invalid syntax


In [2]:
from typing import Dict, List, Tuple, Any

In [3]:
from train_utils import *

In [4]:
from getsecret import getsecret
import subprocess
import json

In [5]:
import jsonmemo as jsonmemo_module
jsonmemo_funcs = jsonmemo_module.create_jsonmemo_funcs(getsecret('DATA_DUMP'))
jsonmemo1arg = jsonmemo_funcs['jsonmemo1arg']
jsonmemo = jsonmemo_funcs['jsonmemo']

In [6]:
def get_models_currently_training():
  output = []
  local_info = json.load(open('current.json'))
  output.append(local_info['base_path'])
  remote_info = json.loads(subprocess.check_output(getsecret('REMOTE_SSH_COMMAND'), shell=True))
  output.append(remote_info['base_path'])
  return output

In [7]:
from glob import glob
import os
from os import path

In [8]:
def find_last_completed_epoch(model_name):
  return len(glob(path.join(model_name, 'info_*.json')))


In [9]:
#print(get_models_with_training_status())


In [10]:


def get_models_with_training_status():
  output = []
  models_currently_training = set(get_models_currently_training())
  for x in glob('tm_*'):
    model_status = {
      'name': x,
      'epochs_done': find_last_completed_epoch(x),
      'status': 'incomplete'
    }
    status_file_path = path.join(x, 'status.json')
    if not os.path.exists(status_file_path):
      output.append(model_status)
      continue
    status_info = json.load(open(status_file_path))
    if status_info['status'] == 'done':
      #model_status['epochs_done'] = find_last_completed_epoch(x)
      model_status['status'] = status_info['status']
      output.append(model_status)
      continue
    if status_info['status'] == 'training':
      if x in models_currently_training:
        model_status['status'] = status_info['status']
      output.append(model_status)
      continue
    raise Exception('invalid status for model', x, status_info['status'])
    #output.append(model_status)
  return output

In [11]:
import shutil

def remove_incomplete_models():
  for status_info in get_models_with_training_status():
    if status_info['epochs_done'] == 0 and status_info['status'] == 'incomplete':
      model_name = status_info['name']
      shutil.rmtree(model_name)
      #print(status_info)

remove_incomplete_models()

In [12]:
def get_parameter_info_list_for_model(model_name):
  parameters_file_path = path.join(model_name, 'parameters.json')
  return json.load(open(parameters_file_path))

def get_parameter_map_for_model(model_name):
  output = {}
  for parameter_info in get_parameter_info_list_for_model(model_name):
    parameter_name = parameter_info['name']
    parameter_value = parameter_info['value']
    output[parameter_name] = parameter_value
  return output

def list_models_finished_training():
  output = []
  for x in glob('tm_*'):
    status_file_path = path.join(x, 'status.json')
    if not os.path.exists(status_file_path):
      continue
    status_info = json.load(open(status_file_path))
    #print(status_info)
    if status_info['status'] != 'done':
      continue
    output.append(x)
  return output

def list_models_matching_parameters(parameters_map):
  output = []
  for model_name in list_models_finished_training():
    parameters_map_cur = get_parameter_map_for_model(model_name)
    is_match = True
    for k,v in parameters_map.items():
      if parameters_map_cur.get(k, None) != v:
        is_match = False
        break
    if is_match:
      output.append(model_name)
  return output


def get_models_with_dev_correct_fraction():
  #for x in list_models_finished_training():
  #  get_parameter_map_for_model(x))
  output = []
  for model_name in list_models_matching_parameters({'enable_current_difficulty': False}):
    for info_file_name in glob(path.join(model_name, 'info_*.json')):
      model_info = json.load(open(info_file_name))
      #print(model_info)
      dev_correct = model_info['dev_correct']
      dev_total = model_info['dev_total']
      dev_correct_fraction = dev_correct / dev_total
      output.append([dev_correct_fraction, info_file_name])
  output.sort()
  output.reverse()
  return output

get_models_with_dev_correct_fraction()

[[0.8085924439986627,
  'tm_5c89f565babee6f8bbc1f7e4d0c76ca4df515adc/info_3.json'],
 [0.8037445670344366,
  'tm_3162e050299b7913eb1388e38b98d6112979995b/info_3.json'],
 [0.8037445670344366,
  'tm_2769148d0b782749562fe2e53e40214082a280c9/info_3.json'],
 [0.7990638582413908,
  'tm_264bddc248d1bba4b2bbdd464ea404764edd3393/info_3.json'],
 [0.7982280173854898,
  'tm_264bddc248d1bba4b2bbdd464ea404764edd3393/info_2.json'],
 [0.7980608492143096,
  'tm_7c371d2a549d40cf966421c94d7d05624b6511a6/info_3.json'],
 [0.797559344700769,
  'tm_264bddc248d1bba4b2bbdd464ea404764edd3393/info_1.json'],
 [0.795218990304246,
  'tm_321abf813a954e3267b1b2665c28a3846bb346fb/info_3.json'],
 [0.7937144767636242,
  'tm_4952aae10684ba73eadb12d1375c6367f6dd799a/info_3.json'],
 [0.7925442995653628,
  'tm_0662ad7f7ca2cc326dad8874568410bbda05bb0c/info_2.json'],
 [0.7908726178535607,
  'tm_0662ad7f7ca2cc326dad8874568410bbda05bb0c/info_1.json'],
 [0.7890337679705784,
  'tm_68170bbdf34ee66979d014128aeecb745062a7c2/info_3.js

In [13]:
# true = True
# false = False
# parameter_info_list : List[Dict[str, Any]] = [{"name": "dataset_name", "type": "dataset", "values": ["2019_04_01"], "value": "2019_04_01"}, {"name": "model_name", "type": "model", "values": ["selfattentionlstm"], "value": "selfattentionlstm"}, {"name": "criterion", "type": "model", "values": ["NLLLoss"], "value": "NLLLoss"}, {"name": "learning_rate", "type": "model", "values": [0.005, 0.05, 0.0005, 5e-05], "value": 5e-05}, {"name": "window_embed_size", "type": "model", "values": [64, 128, 256, 512], "value": 256}, {"name": "difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "time_of_day", "type": "feature", "values": [true, false], "value": true}, {"name": "day_of_week", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_productivity", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_category", "type": "feature", "values": [true, false], "value": true}, {"name": "initial_difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "languages", "type": "feature", "values": [true, false], "value": true}, {"name": "num_prior_entries", "type": "dataparam", "values": [10, 20, 30, 40], "value": 10}, {"name": "sample_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "sample_difficulty_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "disable_prior_visit_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "disable_difficulty_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "enable_current_difficulty", "type": "dataparam", "values": [false, true], "value": false}, {"name": "num_features", "type": "model", "values": [277], "value": 277}]

# train_data,dev_data,test_data = get_data_for_parameters(parameter_info_list)



In [14]:


#def get_current_difficulty(tensor):
#  return tensor['chosen_difficulty']
# will actually always return nothing unless we enabled it in training data
#   feature = tensor['feature']
#   history_length = feature.size()[0]
#   timestep = history_length - 1
#   difficulty_feature_idx = get_feature_index('difficulty')
#   for difficulty_idx,difficulty in enumerate(['nothing', 'easy', 'medium', 'hard']):
#     if feature[timestep][0][difficulty_feature_idx + difficulty_idx].item() > 0:
#       return difficulty
#   return 'nothing'

def get_most_recently_chosen_difficulty(feature, parameter_info_list):
  #feature = tensor['feature']
  history_length = feature.size()[0]
  timestep = history_length - 2
  if len(feature) < 2:
    return 'nothing'
  enabled_feature_list = get_enabled_features_list_from_parameter_info_list(parameter_info_list)
  get_feature_index = make_get_index(enabled_feature_list)
  difficulty_feature_idx = get_feature_index('difficulty')
  for difficulty_idx,difficulty in enumerate(['nothing', 'easy', 'medium', 'hard']):
    if feature[timestep][0][difficulty_feature_idx + difficulty_idx].item() > 0:
      return difficulty
  return 'nothing'

def get_first_chosen_difficulty(feature, parameter_info_list):
  #feature = tensor['feature']
  history_length = feature.size()[0]
  timestep = 0
  if len(feature) < 2:
    return 'nothing'
  enabled_feature_list = get_enabled_features_list_from_parameter_info_list(parameter_info_list)
  get_feature_index = make_get_index(enabled_feature_list)
  difficulty_feature_idx = get_feature_index('difficulty')
  for difficulty_idx,difficulty in enumerate(['nothing', 'easy', 'medium', 'hard']):
    if feature[timestep][0][difficulty_feature_idx + difficulty_idx].item() > 0:
      return difficulty
  return 'nothing'

def get_initial_difficulty(feature, parameter_info_list):
  #feature = tensor['feature']
  history_length = feature.size()[0]
  timestep = 0
  #if len(feature) < 2:
  #  return 'nothing'
  enabled_feature_list = get_enabled_features_list_from_parameter_info_list(parameter_info_list)
  get_feature_index = make_get_index(enabled_feature_list)
  difficulty_feature_idx = get_feature_index('initial_difficulty')
  for difficulty_idx,difficulty in enumerate(['nothing', 'easy', 'medium', 'hard']):
    if feature[timestep][0][difficulty_feature_idx + difficulty_idx].item() > 0:
      return difficulty
  return 'nothing'

def always_predict_nothing(feature, parameter_info_list):
  return 'nothing'

def most_common_key_for_counter(c):
  best_key = None
  for k,v in c.items():
    if best_key == None:
      best_key = k
      continue
    if c[k] > c[best_key]:
      best_key = k
  return best_key

def always_predict_most_common_previous(feature, parameter_info_list):
  history_length = feature.size()[0]
  if len(feature) < 2:
    return 'nothing'
  enabled_feature_list = get_enabled_features_list_from_parameter_info_list(parameter_info_list)
  get_feature_index = make_get_index(enabled_feature_list)
  difficulty_feature_idx = get_feature_index('difficulty')
  difficulty_choices = Counter()
  for timestep in range(0, history_length - 1):
    for difficulty_idx,difficulty in enumerate(['nothing', 'easy', 'medium', 'hard']):
      if feature[timestep][0][difficulty_feature_idx + difficulty_idx].item() > 0:
        difficulty_choices[difficulty] += 1
  output = most_common_key_for_counter(difficulty_choices)
  if output == None:
    return 'nothing'
  return output
  #return 'nothing'

#print(get_current_difficulty(train_data[299]))
#print(get_most_recently_chosen_difficulty(train_data[299]))
# counts = Counter()
# for x in train_data:
#   current_difficulty = get_most_recently_chosen_difficulty(x['feature'])
#   counts[current_difficulty] += 1
# print(counts)

In [15]:
#print(train_data[0])

In [16]:
def evaluate_function_model_on_dataset(parameter_info_list, func, dataset, prefix='dev_') -> Dict[str, Any]:
  confusion = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
  correct = 0
  total = 0
  for item in dataset:
    category_tensor = item['category']
    feature_tensor = item['feature']
    if feature_tensor.size()[0] == 0:
        continue
    difficulty = tensor_to_difficulty(category_tensor)
    difficulty_idx = get_difficulty_idx(difficulty)
    #predicted_tensor = make_prediction(model, feature_tensor.cuda())
    #predicted_difficulty = prediction_to_difficulty(predicted_tensor)
    predicted_difficulty = func(feature_tensor, parameter_info_list)
    predicted_difficulty_idx = get_difficulty_idx(predicted_difficulty)
    if predicted_difficulty_idx == difficulty_idx:
      correct += 1
    total += 1
    confusion[difficulty_idx][predicted_difficulty_idx] += 1
  return {
    prefix + 'correct': correct,
    prefix + 'total': total,
    prefix + 'confusion': confusion,
  }


In [17]:
def evaluate_baseline_model_on_dataset(parameter_info_list, dataset, prefix='dev_'):
  confusion = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
  correct = 0
  total = 0
  for item in dataset:
    category_tensor = item['category']
    feature_tensor = item['feature']
    if feature_tensor.size()[0] == 0:
        continue
    difficulty = tensor_to_difficulty(category_tensor)
    difficulty_idx = get_difficulty_idx(difficulty)
    #predicted_tensor = make_prediction(model, feature_tensor.cuda())
    #predicted_difficulty = prediction_to_difficulty(predicted_tensor)
    predicted_difficulty = get_most_recently_chosen_difficulty(feature_tensor, parameter_info_list)
    predicted_difficulty_idx = get_difficulty_idx(predicted_difficulty)
    if predicted_difficulty_idx == difficulty_idx:
      correct += 1
    total += 1
    confusion[difficulty_idx][predicted_difficulty_idx] += 1
  return {
    prefix + 'correct': correct,
    prefix + 'total': total,
    prefix + 'confusion': confusion,
  }


In [18]:
#print(evaluate_baseline_model_on_dataset(dev_data))

In [19]:
# every_n_visits_to_dev_evaluation_results : Dict[int, Dict[str, Any]] = {}
# every_n_visits_to_train_evaluation_results : Dict[int, Dict[str, Any]] = {}

In [20]:
# true = True
# false = False
# parameter_info_list : List[Dict[str, Any]] = [{"name": "dataset_name", "type": "dataset", "values": ["2019_04_01"], "value": "2019_04_01"}, {"name": "model_name", "type": "model", "values": ["selfattentionlstm"], "value": "selfattentionlstm"}, {"name": "criterion", "type": "model", "values": ["NLLLoss"], "value": "NLLLoss"}, {"name": "learning_rate", "type": "model", "values": [0.005, 0.05, 0.0005, 5e-05], "value": 5e-05}, {"name": "window_embed_size", "type": "model", "values": [64, 128, 256, 512], "value": 256}, {"name": "difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "time_of_day", "type": "feature", "values": [true, false], "value": true}, {"name": "day_of_week", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_productivity", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_category", "type": "feature", "values": [true, false], "value": true}, {"name": "initial_difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "languages", "type": "feature", "values": [true, false], "value": true}, {"name": "num_prior_entries", "type": "dataparam", "values": [10, 20, 30, 40], "value": 10}, {"name": "sample_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "sample_difficulty_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "disable_prior_visit_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "disable_difficulty_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "enable_current_difficulty", "type": "dataparam", "values": [false, true], "value": false}, {"name": "num_features", "type": "model", "values": [277], "value": 277}]
# train_data,dev_data,test_data = get_data_for_parameters(parameter_info_list)


In [21]:
# initial_difficulty_stats = evaluate_function_model_on_dataset(parameter_info_list, get_initial_difficulty, dev_data)
# print(initial_difficulty_stats)

In [22]:
# first_chosen_difficulty_stats = evaluate_function_model_on_dataset(parameter_info_list, get_first_chosen_difficulty, dev_data)
# print(first_chosen_difficulty_stats)

In [23]:
# always_nothing_stats = evaluate_function_model_on_dataset(parameter_info_list, always_predict_nothing, dev_data)
# print(always_nothing_stats)

In [24]:
#most_common_previous_stats = evaluate_function_model_on_dataset(parameter_info_list, always_predict_most_common_previous, dev_data)
#print(most_common_previous_stats)

In [25]:
@jsonmemo1arg
def get_evaluation_results_for_named_baseline(baseline_name):
  baseline_name_to_func = {
    'initial_difficulty': get_initial_difficulty,
    'first_chosen': get_first_chosen_difficulty,
    'always_nothing': always_predict_nothing,
    'most_common_previous': always_predict_most_common_previous,
  }
  true = True
  false = False
  parameter_info_list : List[Dict[str, Any]] = [{"name": "dataset_name", "type": "dataset", "values": ["2019_04_01"], "value": "2019_04_01"}, {"name": "model_name", "type": "model", "values": ["selfattentionlstm"], "value": "selfattentionlstm"}, {"name": "criterion", "type": "model", "values": ["NLLLoss"], "value": "NLLLoss"}, {"name": "learning_rate", "type": "model", "values": [0.005, 0.05, 0.0005, 5e-05], "value": 5e-05}, {"name": "window_embed_size", "type": "model", "values": [64, 128, 256, 512], "value": 256}, {"name": "difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "time_of_day", "type": "feature", "values": [true, false], "value": true}, {"name": "day_of_week", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_productivity", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_category", "type": "feature", "values": [true, false], "value": true}, {"name": "initial_difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "languages", "type": "feature", "values": [true, false], "value": true}, {"name": "num_prior_entries", "type": "dataparam", "values": [10, 20, 30, 40], "value": 10}, {"name": "sample_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "sample_difficulty_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "disable_prior_visit_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "disable_difficulty_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "enable_current_difficulty", "type": "dataparam", "values": [false, true], "value": false}, {"name": "num_features", "type": "model", "values": [277], "value": 277}]
  train_data,dev_data,test_data = get_data_for_parameters(parameter_info_list)
  baseline_func = baseline_name_to_func[baseline_name]
  return evaluate_function_model_on_dataset(parameter_info_list, baseline_func, dev_data)

In [26]:
# baseline_name_to_stats : Dict[str, Dict[str, Any]] = {}
# name_and_func : List[Tuple[str, Any]] = [
#   ('initial_difficulty', get_initial_difficulty),
#   ('first_chosen', get_first_chosen_difficulty),
#   ('always_nothing', always_predict_nothing),
#   ('most_common_previous', always_predict_most_common_previous),
# ]
# for [baseline_name,baseline_func] in name_and_func:
#   baseline_name_to_stats[baseline_name] = evaluate_function_model_on_dataset(parameter_info_list, baseline_func, dev_data)

In [27]:
#print(len(get_users()))

In [28]:
#print(len(get_all_features_data()))

In [ ]:
def set_parameter_in_parameter_info_list(parameter_info_list, parameter_name, parameter_value):
  for parameter_info in parameter_info_list:
    if parameter_info['name'] == parameter_name:
      parameter_info['values'] = [parameter_value]
      parameter_info['value'] = parameter_value

@jsonmemo1arg
def get_evaluation_results_for_sample_every_n_visits(sample_every_n_visits):
  true = True
  false = False
  parameter_info_list = [{"name": "dataset_name", "type": "dataset", "values": ["2019_04_08"], "value": "2019_04_01"}, {"name": "model_name", "type": "model", "values": ["selfattentionlstm"], "value": "selfattentionlstm"}, {"name": "criterion", "type": "model", "values": ["NLLLoss"], "value": "NLLLoss"}, {"name": "learning_rate", "type": "model", "values": [0.005, 0.05, 0.0005, 5e-05], "value": 5e-05}, {"name": "window_embed_size", "type": "model", "values": [64, 128, 256, 512], "value": 256}, {"name": "difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "time_of_day", "type": "feature", "values": [true, false], "value": true}, {"name": "day_of_week", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_productivity", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_category", "type": "feature", "values": [true, false], "value": true}, {"name": "initial_difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "languages", "type": "feature", "values": [true, false], "value": true}, {"name": "num_prior_entries", "type": "dataparam", "values": [10, 20, 30, 40], "value": 10}, {"name": "sample_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "sample_difficulty_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "disable_prior_visit_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "disable_difficulty_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "enable_current_difficulty", "type": "dataparam", "values": [false, true], "value": false}, {"name": "num_features", "type": "model", "values": [277], "value": 277}]
  set_parameter_in_parameter_info_list(parameter_info_list, 'sample_every_n_visits', sample_every_n_visits)
  train_data,dev_data,test_data = get_data_for_parameters(parameter_info_list)
  evaluation_results = evaluate_baseline_model_on_dataset(parameter_info_list, dev_data)
  return evaluation_results

def main():
  for baseline_name in ['initial_difficulty', 'first_chosen', 'always_nothing', 'most_common_previous']:
    print(baseline_name)
    print(get_evaluation_results_for_named_baseline(baseline_name))
  
  sample_every_n_visits_options = [1,2,3,4,5,7,9,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,105,110,115,120,125,130]
  sample_every_n_visits_options.extend(list(range(1, 1000)))
  for sample_every_n_visits in sample_every_n_visits_options:
    print(sample_every_n_visits)
    print(get_evaluation_results_for_sample_every_n_visits(sample_every_n_visits))


main()

initial_difficulty
{'initial_difficulty': {'dev_correct': 2565, 'dev_total': 6439, 'dev_confusion': [[191, 1416, 586, 553], [280, 1756, 409, 143], [14, 209, 514, 59], [4, 23, 178, 104]]}}
first_chosen
{'initial_difficulty': {'dev_correct': 2565, 'dev_total': 6439, 'dev_confusion': [[191, 1416, 586, 553], [280, 1756, 409, 143], [14, 209, 514, 59], [4, 23, 178, 104]]}, 'first_chosen': {'dev_correct': 4721, 'dev_total': 6439, 'dev_confusion': [[2081, 425, 120, 120], [310, 1979, 242, 57], [108, 155, 502, 31], [59, 18, 73, 159]]}}
always_nothing
{'initial_difficulty': {'dev_correct': 2565, 'dev_total': 6439, 'dev_confusion': [[191, 1416, 586, 553], [280, 1756, 409, 143], [14, 209, 514, 59], [4, 23, 178, 104]]}, 'first_chosen': {'dev_correct': 4721, 'dev_total': 6439, 'dev_confusion': [[2081, 425, 120, 120], [310, 1979, 242, 57], [108, 155, 502, 31], [59, 18, 73, 159]]}, 'always_nothing': {'dev_correct': 2746, 'dev_total': 6439, 'dev_confusion': [[2746, 0, 0, 0], [2588, 0, 0, 0], [796, 0, 0,

running split_into_train_dev_test
done with computation 2019_04_08/get_evaluation_results_for_sample_every_n_visits/7.json
{1: {'dev_correct': 5310, 'dev_total': 6439, 'dev_confusion': [[2426, 218, 45, 57], [294, 2158, 114, 22], [110, 99, 549, 38], [70, 19, 43, 177]]}, 2: {'dev_correct': 5228, 'dev_total': 6439, 'dev_confusion': [[2376, 246, 57, 67], [293, 2145, 134, 16], [114, 105, 537, 40], [72, 20, 47, 170]]}, 3: {'dev_correct': 5189, 'dev_total': 6439, 'dev_confusion': [[2369, 243, 65, 69], [292, 2112, 154, 30], [114, 95, 545, 42], [76, 12, 58, 163]]}, 4: {'dev_correct': 5092, 'dev_total': 6439, 'dev_confusion': [[2313, 291, 67, 75], [307, 2084, 168, 29], [107, 123, 529, 37], [62, 23, 58, 166]]}, 5: {'dev_correct': 5044, 'dev_total': 6439, 'dev_confusion': [[2254, 335, 78, 79], [319, 2104, 138, 27], [116, 128, 515, 37], [60, 23, 55, 171]]}, 7: {'dev_correct': 4952, 'dev_total': 6439, 'dev_confusion': [[2218, 336, 83, 109], [295, 2048, 198, 47], [117, 127, 519, 33], [58, 17, 67, 167

running split_into_train_dev_test
done with computation 2019_04_08/get_evaluation_results_for_sample_every_n_visits/20.json
{1: {'dev_correct': 5310, 'dev_total': 6439, 'dev_confusion': [[2426, 218, 45, 57], [294, 2158, 114, 22], [110, 99, 549, 38], [70, 19, 43, 177]]}, 2: {'dev_correct': 5228, 'dev_total': 6439, 'dev_confusion': [[2376, 246, 57, 67], [293, 2145, 134, 16], [114, 105, 537, 40], [72, 20, 47, 170]]}, 3: {'dev_correct': 5189, 'dev_total': 6439, 'dev_confusion': [[2369, 243, 65, 69], [292, 2112, 154, 30], [114, 95, 545, 42], [76, 12, 58, 163]]}, 4: {'dev_correct': 5092, 'dev_total': 6439, 'dev_confusion': [[2313, 291, 67, 75], [307, 2084, 168, 29], [107, 123, 529, 37], [62, 23, 58, 166]]}, 5: {'dev_correct': 5044, 'dev_total': 6439, 'dev_confusion': [[2254, 335, 78, 79], [319, 2104, 138, 27], [116, 128, 515, 37], [60, 23, 55, 171]]}, 7: {'dev_correct': 4952, 'dev_total': 6439, 'dev_confusion': [[2218, 336, 83, 109], [295, 2048, 198, 47], [117, 127, 519, 33], [58, 17, 67, 16

running split_into_train_dev_test
done with computation 2019_04_08/get_evaluation_results_for_sample_every_n_visits/35.json
{1: {'dev_correct': 5310, 'dev_total': 6439, 'dev_confusion': [[2426, 218, 45, 57], [294, 2158, 114, 22], [110, 99, 549, 38], [70, 19, 43, 177]]}, 2: {'dev_correct': 5228, 'dev_total': 6439, 'dev_confusion': [[2376, 246, 57, 67], [293, 2145, 134, 16], [114, 105, 537, 40], [72, 20, 47, 170]]}, 3: {'dev_correct': 5189, 'dev_total': 6439, 'dev_confusion': [[2369, 243, 65, 69], [292, 2112, 154, 30], [114, 95, 545, 42], [76, 12, 58, 163]]}, 4: {'dev_correct': 5092, 'dev_total': 6439, 'dev_confusion': [[2313, 291, 67, 75], [307, 2084, 168, 29], [107, 123, 529, 37], [62, 23, 58, 166]]}, 5: {'dev_correct': 5044, 'dev_total': 6439, 'dev_confusion': [[2254, 335, 78, 79], [319, 2104, 138, 27], [116, 128, 515, 37], [60, 23, 55, 171]]}, 7: {'dev_correct': 4952, 'dev_total': 6439, 'dev_confusion': [[2218, 336, 83, 109], [295, 2048, 198, 47], [117, 127, 519, 33], [58, 17, 67, 16

running split_into_train_dev_test
done with computation 2019_04_08/get_evaluation_results_for_sample_every_n_visits/50.json
{1: {'dev_correct': 5310, 'dev_total': 6439, 'dev_confusion': [[2426, 218, 45, 57], [294, 2158, 114, 22], [110, 99, 549, 38], [70, 19, 43, 177]]}, 2: {'dev_correct': 5228, 'dev_total': 6439, 'dev_confusion': [[2376, 246, 57, 67], [293, 2145, 134, 16], [114, 105, 537, 40], [72, 20, 47, 170]]}, 3: {'dev_correct': 5189, 'dev_total': 6439, 'dev_confusion': [[2369, 243, 65, 69], [292, 2112, 154, 30], [114, 95, 545, 42], [76, 12, 58, 163]]}, 4: {'dev_correct': 5092, 'dev_total': 6439, 'dev_confusion': [[2313, 291, 67, 75], [307, 2084, 168, 29], [107, 123, 529, 37], [62, 23, 58, 166]]}, 5: {'dev_correct': 5044, 'dev_total': 6439, 'dev_confusion': [[2254, 335, 78, 79], [319, 2104, 138, 27], [116, 128, 515, 37], [60, 23, 55, 171]]}, 7: {'dev_correct': 4952, 'dev_total': 6439, 'dev_confusion': [[2218, 336, 83, 109], [295, 2048, 198, 47], [117, 127, 519, 33], [58, 17, 67, 16

running split_into_train_dev_test
done with computation 2019_04_08/get_evaluation_results_for_sample_every_n_visits/65.json
{1: {'dev_correct': 5310, 'dev_total': 6439, 'dev_confusion': [[2426, 218, 45, 57], [294, 2158, 114, 22], [110, 99, 549, 38], [70, 19, 43, 177]]}, 2: {'dev_correct': 5228, 'dev_total': 6439, 'dev_confusion': [[2376, 246, 57, 67], [293, 2145, 134, 16], [114, 105, 537, 40], [72, 20, 47, 170]]}, 3: {'dev_correct': 5189, 'dev_total': 6439, 'dev_confusion': [[2369, 243, 65, 69], [292, 2112, 154, 30], [114, 95, 545, 42], [76, 12, 58, 163]]}, 4: {'dev_correct': 5092, 'dev_total': 6439, 'dev_confusion': [[2313, 291, 67, 75], [307, 2084, 168, 29], [107, 123, 529, 37], [62, 23, 58, 166]]}, 5: {'dev_correct': 5044, 'dev_total': 6439, 'dev_confusion': [[2254, 335, 78, 79], [319, 2104, 138, 27], [116, 128, 515, 37], [60, 23, 55, 171]]}, 7: {'dev_correct': 4952, 'dev_total': 6439, 'dev_confusion': [[2218, 336, 83, 109], [295, 2048, 198, 47], [117, 127, 519, 33], [58, 17, 67, 16

In [26]:
# sample_every_n_visits_options : List[int] = list(range(1, 1000))

# true = True
# false = False
# for sample_every_n_visits in sample_every_n_visits_options:
#   if sample_every_n_visits in every_n_visits_to_dev_evaluation_results:
#     continue
#   parameter_info_list = [{"name": "dataset_name", "type": "dataset", "values": ["2019_04_08"], "value": "2019_04_01"}, {"name": "model_name", "type": "model", "values": ["selfattentionlstm"], "value": "selfattentionlstm"}, {"name": "criterion", "type": "model", "values": ["NLLLoss"], "value": "NLLLoss"}, {"name": "learning_rate", "type": "model", "values": [0.005, 0.05, 0.0005, 5e-05], "value": 5e-05}, {"name": "window_embed_size", "type": "model", "values": [64, 128, 256, 512], "value": 256}, {"name": "difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "time_of_day", "type": "feature", "values": [true, false], "value": true}, {"name": "day_of_week", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_productivity", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_category", "type": "feature", "values": [true, false], "value": true}, {"name": "initial_difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "languages", "type": "feature", "values": [true, false], "value": true}, {"name": "num_prior_entries", "type": "dataparam", "values": [10, 20, 30, 40], "value": 10}, {"name": "sample_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "sample_difficulty_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "disable_prior_visit_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "disable_difficulty_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "enable_current_difficulty", "type": "dataparam", "values": [false, true], "value": false}, {"name": "num_features", "type": "model", "values": [277], "value": 277}]
#   for parameter_info in parameter_info_list:
#     if parameter_info['name'] == 'sample_every_n_visits':
#       parameter_info['values'] = [sample_every_n_visits]
#       parameter_info['value'] = sample_every_n_visits
#   train_data,dev_data,test_data = get_data_for_parameters(parameter_info_list)
#   print(sample_every_n_visits)
#   evaluation_results = evaluate_baseline_model_on_dataset(parameter_info_list, dev_data)
#   every_n_visits_to_dev_evaluation_results[sample_every_n_visits] = evaluation_results
#   print(evaluation_results)
#   #evaluation_results = evaluate_baseline_model_on_dataset(parameter_info_list, train_data)
#   #every_n_visits_to_train_evaluation_results[sample_every_n_visits] = evaluation_results
#   #print(evaluation_results)





{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
1
{'dev_correct': 5310, 'dev_total': 6439, 'dev_confusion': [[2426, 218, 45, 57], [294, 2158, 114, 22], [110, 99, 549, 38], [70, 19, 43, 177]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [2], 'value': 2}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_p

12
{'dev_correct': 4793, 'dev_total': 6439, 'dev_confusion': [[2149, 367, 113, 117], [300, 1979, 246, 63], [108, 140, 507, 41], [62, 15, 74, 158]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [13], 'value': 13}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
13
{'dev_correct': 4753, 'dev_total': 6439, 'dev_confusion': [[2145, 350, 129, 122], [330, 1944, 258, 56], [113, 142, 516, 25], [73, 15, 73, 148]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type'

24
{'dev_correct': 4363, 'dev_total': 6439, 'dev_confusion': [[1898, 505, 168, 175], [318, 1860, 321, 89], [113, 172, 473, 38], [66, 13, 98, 132]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [25], 'value': 25}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
25
{'dev_correct': 4249, 'dev_total': 6439, 'dev_confusion': [[1797, 578, 168, 203], [331, 1848, 318, 91], [114, 179, 462, 41], [50, 16, 101, 142]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type

36
{'dev_correct': 4000, 'dev_total': 6439, 'dev_confusion': [[1666, 627, 233, 220], [325, 1758, 406, 99], [115, 195, 460, 26], [58, 16, 119, 116]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [37], 'value': 37}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
37
{'dev_correct': 3995, 'dev_total': 6439, 'dev_confusion': [[1643, 648, 231, 224], [321, 1781, 387, 99], [114, 199, 456, 27], [58, 16, 120, 115]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'typ

48
{'dev_correct': 3856, 'dev_total': 6439, 'dev_confusion': [[1567, 695, 260, 224], [338, 1739, 410, 101], [114, 209, 445, 28], [65, 18, 121, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [49], 'value': 49}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
49
{'dev_correct': 3837, 'dev_total': 6439, 'dev_confusion': [[1551, 707, 262, 226], [339, 1737, 411, 101], [114, 210, 444, 28], [65, 18, 121, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 't

...

948
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [949], 'value': 949}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
949
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits'

960
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [961], 'value': 961}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
961
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits'

972
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [973], 'value': 973}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
973
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits'

984
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [985], 'value': 985}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
985
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits'

996
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [997], 'value': 997}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
997
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits'

In [ ]:
scatterplot = []
for k,v in every_n_visits_to_dev_evaluation_results.items():
  scatterplot.append([k,v['dev_correct'] / v['dev_total']])

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


In [ ]:
print(scatterplot)

In [ ]:


# Create a trace
trace = go.Scatter(
    x = [x[0] for x in scatterplot],
    y = [x[1] for x in scatterplot],
    mode = 'markers'
)

data = [trace]

# Plot and embed in ipython notebook!
iplot(data)

In [ ]:
baseline_name_and_correctness : List[Tuple[str, float]] = []
for k2,v2 in baseline_name_to_stats.items():
  correctness : float = float(v2['dev_correct'] / v2['dev_total'])
  baseline_name_and_correctness.append((k2, correctness))
print(baseline_name_and_correctness)